In [1]:
Dataset = "FoCus"                      # "Blended Skill Talk", "IT-ConvAI2" , "FoCus" 

_COT = ""                                   #  "",    "-COT"

SCORING_METHOD = "avg"                      # "avg",  "length_prior"

LLM_name = f"Qwen2-5B-{Dataset}-{SCORING_METHOD}"

In [2]:
LLM_name

'Qwen2-5B-FoCus-avg'

In [3]:
import pandas as pd

ds = pd.read_csv(f'Raw Responses/{Dataset}/{LLM_name}{_COT}.csv')

print("Shape:", ds.shape)

print("\nMissing Values:\n", ds.isnull().sum())

ds

Shape: (1000, 2)

Missing Values:
 gen_response     0
response_time    0
dtype: int64


,gen_response,response_time
0,"{\n ""response"": ""I'm sorry, but I don't have ...",0.902046
1,"{\n ""response"": ""I'm sorry, but I don't see a...",0.941635
2,"{\n ""response"": ""Great choice for your next v...",0.887553
3,"{\n ""response"": ""Great choice! Technische Uni...",0.894501
4,"{\n ""response"": ""I'm sorry, but I don't under...",0.884114
...,...,...
995,"{\n ""response"": ""I'm sorry, but as a particip...",0.889212
996,"{\n ""response"": ""I'm sorry, but I don't under...",0.884870
997,"{\n ""response"": ""To learn more about history,...",0.889985
998,"{\n ""response"": ""Based on the conversation co...",0.882518


In [4]:
print(ds.iloc[167]["gen_response"])

{
  "response": "I'm sorry, but I don't have any personal experience with the topic you're asking about. However, I can tell you that Thursday Island Cemetery is a fascinating place for anyone interested in history and cultural significance. The cemetery's collection of Japanese graves adds to its historical value, as many Japanese divers were involved in finding pearls. Visitors can explore the grounds and learn about the island's rich past through interactive exhibits and guided tours. Overall, it's definitely worth visiting if you're interested in exploring the history and beauty of


In [5]:
import re
import json
import pandas as pd

# Define functions
def find_first_valid_json(text):

    json_objects = re.findall(r'\{.*?\}', text, re.DOTALL)
    for obj in json_objects:
        try:
            json_obj = json.loads(obj)
            if "response" in json_obj:  # Only check for "response"
                return json_obj
        except json.JSONDecodeError:
            continue
    return None

def get_response(text):
    if text is not None:  # Check if text is not None
        try:
            return text['response']
        except (ValueError, SyntaxError, KeyError):
            return None
    return None



# Replace empty strings in 'gen_response' with None
ds.loc[ds['gen_response'] == '', 'gen_response'] = None

# Apply the find_first_valid_json function
ds['gen_response'] = ds['gen_response'].apply(lambda x: find_first_valid_json(x))

# Convert gen_response to None if it's not a valid string
ds['gen_response'] = ds['gen_response'].apply(lambda x: None if pd.isna(x) or x == 'nan' or isinstance(x, float) else x)

# Extract 'response' from the JSON objects
ds['gen_response'] = ds['gen_response'].apply(lambda x: get_response(x))

# Keep the 'response_time' column unchanged
ds['response_time'] = ds['response_time']

# Define the new column order
new_column_order = ['gen_response', 'response_time']

# Reorder the columns
ds = ds[new_column_order]

print("\nMissing Values:\n", ds.isnull().sum())
print(ds.shape)
ds



Missing Values:
 gen_response     383
response_time      0
dtype: int64
(1000, 2)


,gen_response,response_time
0,"I'm sorry, but I don't have any personal exper...",0.902046
1,"I'm sorry, but I don't see any specific inform...",0.941635
2,Great choice for your next visit! The Marion P...,0.887553
3,None,0.894501
4,"I'm sorry, but I don't understand what you're ...",0.884114
...,...,...
995,"I'm sorry, but as a participant in this conver...",0.889212
996,"I'm sorry, but I don't understand what you're ...",0.884870
997,None,0.889985
998,None,0.882518


In [6]:
ds.to_csv(f'Responses/{Dataset}/{LLM_name}{_COT}.csv', index=False)